# SQL - финальный проект

## Цели
Подключиться к БД SQL postgres 

Выполнить задания используя запросы SQL, вывести ответы в pandas

## Описание
БД содержит информацию о крупном сервисе  для чтения книг по подписке (книгах, издательствах, авторах, а также пользовательские
обзоры книг). 

## Анализ БД

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [3]:
#функция для вывода информации о таблице 
def db(query):
    table = pd.io.sql.read_sql(query, con=engine)
    return display(table.head()), table.info()

#функция для решения задачи
def answer(query):
    return pd.io.sql.read_sql(query, con=engine)

### books

In [4]:
# выгружаем бд полностью для анализа
query = ''' SELECT *
            FROM books
        '''
books = pd.io.sql.read_sql(query, con=engine, index_col='book_id')

display(books.head())
books.info()

,author_id,title,num_pages,publication_date,publisher_id
book_id,,,,,
1,546,'Salem's Lot,594,2005-11-01,93
2,465,1 000 Places to See Before You Die,992,2003-05-22,336
3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 1 to 1000
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   author_id         1000 non-null   int64 
 1   title             1000 non-null   object
 2   num_pages         1000 non-null   int64 
 3   publication_date  1000 non-null   object
 4   publisher_id      1000 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 46.9+ KB


В books 1000 уникальных книг

### authors

In [5]:
query = ''' SELECT *
            FROM authors
        '''
authors = pd.io.sql.read_sql(query, con=engine, index_col='author_id')

display(authors.head())
authors.info()

,author
author_id,
1,A.S. Byatt
2,Aesop/Laura Harris/Laura Gibbs
3,Agatha Christie
4,Alan Brennert
5,Alan Moore/David Lloyd


<class 'pandas.core.frame.DataFrame'>
Int64Index: 636 entries, 1 to 636
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   author  636 non-null    object
dtypes: object(1)
memory usage: 9.9+ KB


В authors 636 уникальных авторов. Проверим, что столько же и в books (понадобится, когда будем соединять таблицы)

In [6]:
books['author_id'].nunique()

636

In [7]:
#ту же самую проверку можно сделать в SQL, не выгружая всю таблицу в pandas
query = ''' SELECT COUNT(DISTINCT(author_id)) AS authors_count
            FROM books
        '''
answer(query)

,authors_count
0,636


### ratings

In [8]:
query = ''' SELECT *
            FROM ratings
        '''

ratings = pd.io.sql.read_sql(query, con=engine, index_col='rating_id')

display(ratings.head())
ratings.info()

,book_id,username,rating
rating_id,,,
1,1,ryanfranco,4
2,1,grantpatricia,2
3,1,brandtandrea,5
4,2,lorichen,3
5,2,mariokeller,2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6456 entries, 1 to 6456
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   book_id   6456 non-null   int64 
 1   username  6456 non-null   object
 2   rating    6456 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 201.8+ KB


In [9]:
ratings['book_id'].nunique() #уникальных книг столько же, сколько и в books

1000

In [10]:
#ту же самую проверку можно сделать в SQL, не выгружая всю таблицу в pandas
query = ''' SELECT COUNT(DISTINCT(book_id)) AS book_count
            FROM ratings
        '''

book_count = pd.io.sql.read_sql(query, con=engine)

book_count

,book_count
0,1000


In [11]:
#все книги в ratings из таблицы books
ratings['book_id'].isin(books.index).value_counts()

True    6456
Name: book_id, dtype: int64

В rating 6456 уникальных оценок. Книги теже самые, что и в books

### reviews

In [12]:
query = ''' SELECT *
            FROM reviews
        '''

reviews = pd.io.sql.read_sql(query, con=engine, index_col='review_id')

display(reviews.head())
reviews.info()

,book_id,username,text
review_id,,,
1,1,brandtandrea,Mention society tell send professor analysis. ...
2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
4,3,johnsonamanda,Finally month interesting blue could nature cu...
5,3,scotttamara,Nation purpose heavy give wait song will. List...


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2793 entries, 1 to 2793
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   book_id   2793 non-null   int64 
 1   username  2793 non-null   object
 2   text      2793 non-null   object
dtypes: int64(1), object(2)
memory usage: 87.3+ KB


In [13]:
reviews['book_id'].nunique()

994

In [14]:
#ту же самую проверку можно сделать в SQL, не выгружая всю таблицу в pandas
query = ''' SELECT COUNT(DISTINCT(book_id)) AS book_count
            FROM reviews
        '''

book_count = pd.io.sql.read_sql(query, con=engine)

book_count

,book_count
0,994


In [15]:
reviews['book_id'].isin(books.index).value_counts()

True    2793
Name: book_id, dtype: int64

В reviews 2793 отзывов. Книг 994 - т.е. часть книг (1000-994=6) не попала таблицу reviews. Ниже id книг, которые есть в таблицах books, ratings, но нет в reviews. 

In [16]:
#ту же самую проверку можно сделать в SQL, не выгружая всю таблицу в pandas
query = ''' SELECT book_id
            FROM reviews
            WHERE book_id NOT IN (SELECT book_id
                                   FROM books)
        '''

books_wo_reviews = pd.io.sql.read_sql(query, con=engine)

books_wo_reviews

,book_id


In [17]:
#ту же самую проверку можно сделать в SQL, не выгружая всю таблицу в pandas
query = ''' SELECT book_id
            FROM books
            WHERE book_id NOT IN (SELECT book_id
                                   FROM reviews)
        '''

books_wo_reviews = pd.io.sql.read_sql(query, con=engine)

books_wo_reviews

,book_id
0,83
1,191
2,221
3,387
4,672
5,808


In [18]:
ratings[~ratings['book_id'].isin(reviews['book_id'])]['book_id'].unique()

array([ 83, 191, 221, 387, 672, 808])

In [19]:
#ту же самую проверку можно сделать в SQL, не выгружая всю таблицу в pandas
query = ''' SELECT DISTINCT r.book_id AS books_wo_reviews ---ы
            FROM ratings AS r
            LEFT JOIN reviews AS rev on r.book_id=rev.book_id
            WHERE rev.book_id IS NULL
        '''

books_wo_reviews = pd.io.sql.read_sql(query, con=engine)

books_wo_reviews

,books_wo_reviews
0,83
1,191
2,221
3,387
4,672
5,808


### publishers

In [20]:
query = ''' SELECT *
            FROM publishers
        '''

publishers = pd.io.sql.read_sql(query, con=engine, index_col='publisher_id')

display(publishers.head())
publishers.info()

,publisher
publisher_id,
1,Ace
2,Ace Book
3,Ace Books
4,Ace Hardcover
5,Addison Wesley Publishing Company


<class 'pandas.core.frame.DataFrame'>
Int64Index: 340 entries, 1 to 340
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   publisher  340 non-null    object
dtypes: object(1)
memory usage: 5.3+ KB


In [21]:
books['publisher_id'].nunique()

340

In [22]:
books['publisher_id'].isin(publishers.index).value_counts()

True    1000
Name: publisher_id, dtype: int64

В publishers 340 издательств, те же самые, что и в books

In [23]:
ratings['username'].nunique()

160

In [24]:
reviews['username'].nunique()

160

In [25]:
#из pd.Series c пользователями сделали списки, отсортировали их по алфавиту и сравнили друг с другом. 
list(ratings['username'].unique()).sort() == list(reviews['username'].unique()).sort()
#результат: удостоверились что пользователи оставляющие обзоры и оценки - это те же самые пользователи. 

True

In [26]:
#ту же самую проверку можно сделать в SQL, не выгружая всю таблицу в pandas
query = ''' SELECT COUNT(DISTINCT r.username) AS user_count
            FROM ratings AS r
            INNER JOIN reviews AS rev on r.username=rev.username
        '''

user_count = pd.io.sql.read_sql(query, con=engine)

user_count

,user_count
0,160


## Задание №1
Посчитайте, сколько книг вышло после 1 января 2000 года

In [27]:
query = ''' SELECT COUNT(*)
            FROM books
            WHERE publication_date > DATE '2000-01-01';
        '''
print('после 1 января 2000 года вышло:')
answer(query)

после 1 января 2000 года вышло:


,count
0,819


## Задание №2
Для каждой книги посчитайте количество обзоров и среднюю оценку

In [28]:
query = ''' SELECT r.book_id, 
                   b.title,
                   AVG(r.rating) AS avg_rating, --- средняя оценка
                   COUNT(DISTINCT rev.text) AS review_count --- количество обзоров
            FROM ratings AS r
                 INNER JOIN books AS b ON b.book_id = r.book_id --- объединение ratings и books (чтобы подцепить название книги)
                 LEFT JOIN reviews AS rev ON r.book_id = rev.book_id --- объединние с reviews
            GROUP BY r.book_id, b.title --- группировка по книгам
        '''
answer = pd.io.sql.read_sql(query, con=engine, index_col='book_id')
print('выводим первые строки таблицы с количеством обзоров и средней оценкой книг:')
answer.head()

выводим первые строки таблицы с количеством обзоров и средней оценкой книг:


,title,avg_rating,review_count
book_id,,,
1,'Salem's Lot,3.666667,2
2,1 000 Places to See Before You Die,2.500000,1
3,13 Little Blue Envelopes (Little Blue Envelope...,4.666667,3
4,1491: New Revelations of the Americas Before C...,4.500000,2
5,1776,4.000000,4


In [29]:
answer['review_count'].sum()

2793

In [30]:
reviews.shape[0]

2793

books и rating соединяем INNER JOIN т.к. у них одинаковый набор книг. Reviews к ним присоединяем с помощью LEFT JOIN, чтобы книги, которых нет в reviews не потерялись. 

In [31]:
#проверяем как отобразились книги, которых нет в reviews - у них 0 в количестве обзоров - все корректно.
answer.query('book_id == 83')

,title,avg_rating,review_count
book_id,,,
83,Anne Rice's The Vampire Lestat: A Graphic Novel,3.666667,0


In [32]:
books['publisher_id'].nunique()

340

## Задание 3
Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [33]:
query = ''' SELECT p.publisher,
                   COUNT(b.title) AS book_count --- количество книг, выпущенное издательством
            FROM books AS b
                 INNER JOIN publishers AS p ON b.publisher_id = p.publisher_id 
            WHERE b.num_pages > 50 --- выбираем книги со страницами больше 50
            GROUP BY p.publisher_id --- группировка по издательству
            ORDER BY book_count DESC --- сортировка по убыванию
            LIMIT 1 --- выбираем одно издательство
        '''
print('издательство, выпустившие наибольшее количество книг толще 50 страниц:')
answer = pd.io.sql.read_sql(query, con=engine)
answer

издательство, выпустившие наибольшее количество книг толще 50 страниц:


,publisher,book_count
0,Penguin Books,42


Количество издательств в books и publishers совпадают, поэтому INNER JOIN

In [34]:
books['author_id'].nunique()

636

In [35]:
authors.shape[0]

636

## Задание № 4
Определите автора с самой высокой средней оценкой книг — учитывайте
только книги с 50 и более оценками

In [36]:
query = ''' WITH
            b AS (SELECT *
                FROM books
                WHERE book_id IN (SELECT book_id
                                 FROM ratings 
                                 GROUP BY book_id
                                 HAVING COUNT(rating) >= 50))
            SELECT a.author, 
                   AVG(r.rating) AS book_rating
            FROM b
                 INNER JOIN authors AS a ON b.author_id = a.author_id
                 INNER JOIN ratings as r ON b.book_id = r.book_id
            GROUP BY a.author
            ORDER BY book_rating DESC
            LIMIT 1
        '''
print('Автор с самой высокой средней оценкой книг (учитываются книги только книги с 50 и более оценками):')
answer = pd.io.sql.read_sql(query, con=engine)
answer

Автор с самой высокой средней оценкой книг (учитываются книги только книги с 50 и более оценками):


,author,book_rating
0,J.K. Rowling/Mary GrandPré,4.287097


## Задание 5
Посчитайте среднее количество обзоров от пользователей, которые поставили
больше 50 оценок.

In [37]:
query = '''SELECT AVG(users.reviews_count) --- среднее количество обзоров пользователей 
        FROM 
        (SELECT username, 
                COUNT(text) AS reviews_count
        FROM reviews
        WHERE username IN (SELECT username --- пользователи поставившие больше 50 оценок
                           FROM ratings
                           GROUP by username
                           HAVING COUNT(rating) >= 50)
        GROUP BY username) AS users
        '''
print('среднее количество обзоров от пользователей, которые поставили больше 50 оценок:')
answer = pd.io.sql.read_sql(query, con=engine)
answer

среднее количество обзоров от пользователей, которые поставили больше 50 оценок:


,avg
0,24.222222
